In [ ]:
!pip install -q spacy
!pip install -q tensorflow
!pip install -q transformers

In [ ]:
%env TF_CPP_MIN_LOG_LEVEL = 3

env: TF_CPP_MIN_LOG_LEVEL=3


In [ ]:
import math
import nltk
import spacy
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import transformers
from tokenizers import BertWordPieceTokenizer
from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tensorflow import keras
from keras.utils import to_categorical
from tensorflow.keras import Sequential
from keras.preprocessing.text import Tokenizer
from keras.metrics import Precision, Recall, AUC
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, LearningRateScheduler, CallbackList, ReduceLROnPlateau
from tensorflow.keras.optimizers.experimental import Adam

# **Loading Data**

In [ ]:
# Run only if you are using Google Colab
from google.colab import drive

'''drive.mount('/content/drive')

import os

path = '/content/drive/MyDrive/Colab Notebooks/Cap7''''

path = './datas'

Mounted at /content/drive


In [ ]:
train_data = pd.read_csv(os.path.join(path, 'dados_treino.txt'), header = None, delimiter = ';')

test_data = pd.read_csv(os.path.join(path, 'dados_teste.txt'), header = None, delimiter = ';')

In [ ]:
train_data = train_data.rename(columns = {0: 'text', 1: 'sentiment'})
test_data = test_data.rename(columns = {0: 'text', 1: 'sentiment'})

In [ ]:
train_data.shape

(16000, 2)

In [ ]:
test_data.shape

(2000, 2)

In [ ]:
train_data.head()

,text,sentiment
0,i am feeling completely overwhelmed i have two...,fear
1,i have the feeling she was amused and delighted,joy
2,i was able to help chai lifeline with your sup...,joy
3,i already feel like i fucked up though because...,anger
4,i still love my so and wish the best for him i...,sadness


In [ ]:
train_data['sentiment'].value_counts()

sentiment
joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64

In [ ]:
test_data['sentiment'].value_counts()

sentiment
joy         695
sadness     581
anger       275
fear        224
love        159
surprise     66
Name: count, dtype: int64

# **PreProcessing with Spacy**

In [ ]:
# Download Dict
!python -m spacy download en_core_web_md -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 23.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Loading Dict
nlp_dict = spacy.load('en_core_web_md')

In [ ]:
def preprocessing_text(text):
  # Processing text with Dict
  doc = nlp_dict(text)

  # Create an list with tokens
  tokens = [token.lemma_.lower().strip() for token in doc if not token.is_stop]

  # Return tokens
  return ' '.join(tokens)

In [ ]:
train_data['Processed_text'] = train_data['text'].apply(preprocessing_text)

test_data['Processed_text'] = test_data['text'].apply(preprocessing_text)

In [ ]:
train_data.head()

,text,sentiment,Processed_text
0,i am feeling completely overwhelmed i have two...,fear,feel completely overwhelmed strategy help feel...
1,i have the feeling she was amused and delighted,joy,feeling amuse delight
2,i was able to help chai lifeline with your sup...,joy,able help chai lifeline support encouragement ...
3,i already feel like i fucked up though because...,anger,feel like fuck not usually eat morning
4,i still love my so and wish the best for him i...,sadness,love wish good long tolerate effect bm life fa...


In [ ]:
# Function to encode text into int sequences for Bert model input
def encode(texts, tokenizer, chunk_size = 256, maxlen = 512):
  # Enable truncation in tokenizer to max lenght
  tokenizer.enable_truncation(max_length = maxlen)

  # Enable padding in tokenizer
  tokenizer.enable_padding(length = maxlen)

  all_ids = []

  # Iterate over all text in 'chunk pieces'
  for i in tqdm(range(0, len(texts), chunk_size)):
    text_chunk = texts[i:i+chunk_size].tolist()

    encs = tokenizer.encode_batch(text_chunk)

    # Extemded the list 'all_ids' with encoded Ids
    all_ids.extend(enc.ids for enc in encs)

  return np.array(all_ids)

In [ ]:
# Loading Tokenizer of pre-trained model
bert_tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

In [ ]:
# Saving the tokenizer locally
bert_tokenizer.save_pretrained('.')

('./tokenizer_config.json',
 './special_tokens_map.json',
 './vocab.txt',
 './added_tokens.json')

In [ ]:
# Loading a faster tokenizer using the core tokenizer vocabulary
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase = False)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data['Processed_text'].values,
                                                      train_data['sentiment'].values,
                                                      test_size = 0.2,
                                                      random_state = 42,
                                                      stratify = train_data['sentiment'])

In [ ]:
# Maximum length used in text
max_length = 100

In [ ]:
# Apply tokenization
X_train_encoded = encode(X_train, fast_tokenizer, maxlen = max_length)
X_valid_encoded = encode(X_valid, fast_tokenizer, maxlen = max_length)
X_test_encoded = encode(test_data['Processed_text'], fast_tokenizer, maxlen = max_length)

100%|██████████| 8/8 [00:00<00:00, 52.59it/s]


In [ ]:
label_encoder = LabelEncoder()

y_train_le = label_encoder.fit_transform(y_train)
y_valid_le = label_encoder.transform(y_valid)
y_test_le = label_encoder.transform(test_data['sentiment'])

y_train_encoded = to_categorical(y_train_le)
y_valid_encoded = to_categorical(y_valid_le)
y_test_encoded = to_categorical(y_test_le)

In [ ]:
BATCH_SIZE = 16

In [ ]:
train_dataset = (tf.data.Dataset
                 .from_tensor_slices((X_train_encoded, y_train_encoded))
                 .repeat()
                 .shuffle(2048)
                 .batch(BATCH_SIZE)
                 )

In [ ]:
valid_dataset = (tf.data.Dataset
                 .from_tensor_slices((X_valid_encoded, y_valid_encoded))
                 .batch(BATCH_SIZE)
                 .cache()
                 )

In [ ]:
test_dataset = (tf.data.Dataset
                .from_tensor_slices((X_test_encoded, y_test_encoded))
                .batch(BATCH_SIZE)
                )

# **Create Model**

In [ ]:
# Function to apply fine-tune and train a transformer model (e.g. BERT)
def create_model(transformer, max_len = 512):

  # Define input layer
  input_word_ids = tf.keras.layers.Input(shape = (max_len,),
                                          dtype = tf.int32,
                                          name = 'input_word_ids')

  # Define sequence output of model
  sequence_output = transformer(input_word_ids)[0]

  # Select the first token of each sequence (token CLS in bert) for classification
  cls_token = sequence_output[:, 0, :]

  # Add a layer dense for classifcation
  out = Dense(6, activation = 'softmax')(cls_token)

  # Create keras model with input and output selected
  model = tf.keras.Model(inputs = input_word_ids, outputs = out)

  model.compile(tf.keras.optimizers.legacy.Adam(learning_rate = 1e-5),
                loss = 'categorical_crossentropy',
                metrics = ['accuracy', Precision(), Recall(), AUC()])

  return model


In [ ]:
# Create an instance of pretrained DistilBERT multilingual
transformer_layer = (transformers.TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased'))

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
model = create_model(transformer_layer, max_len = max_length)

In [ ]:
# Freezing pretained layers
model.layers[0].trainable = False
model.layers[1].trainable = False
model.layers[2].trainable = False

In [ ]:
n_steped = X_train_encoded.shape[0] // BATCH_SIZE
num_epochs = 3

In [ ]:
%%time
history = model.fit(train_dataset,
                    steps_per_epoch = n_steps,
                    validation_data = valid_dataset,
                    epochs = num_epochs)

In [ ]:
loss, val_loss = history.history['loss'], history.history['val_loss']
plt.plot(loss, label = 'Loss')
plt.plot(val_loss, label = 'Validation Loss')
plt.legend()
plt.show()

In [ ]:
predict = model.predict(X_test_encoded)

In [ ]:
predicted_labels = predict.argmax(axis = 1)

In [ ]:
print(classification_report(y_test_le, predicted_labels))

In [ ]:
print(confusion_matrix(y_test_le, predicted_labels))

In [ ]:
print(accuracy_score(y_test_le, predicted_labels))

In [ ]:
model.save('./models/model_v3.keras', save_format='tf')

# **Deploy**

In [ ]:
# Loading Model
# Imports
from transformers import TFDistilBertModel
from tensorflow.keras.utils import custom_object_scope

# Register personalized layer and load the model
with custom_object_scope({'TFDistilBertModel': TFDistilBertModel}):
  loaded_model = tf.keras.models.load_model('./models/model_v3.keras')

In [ ]:
phrase = 'i even feel a little shaky'

In [ ]:
df = pd.DataFrame({'Phrase': [phrase]})

In [ ]:
df['Processed_phrase'] = df['Phrase'].apply(preprocessing_text)

In [ ]:
new_data = encode(df['Processed_phrase'], fast_tokenizer, max_len = max_length)

In [ ]:
result = loaded_model.predict(new_data)

In [ ]:
prob = np.argmax(result, axis = 1)

In [ ]:
name_cls = label_encoder.inverse_transform(prob)
name_cls

In [ ]:
%env TOKENIZERS_PARALLELISM = false